# PyTorch Deep Dive: Building Neural Networks

You know Tensors (Data). You know Autograd (Math). Now let's build the **Machine**.

Before we start assembling, we need to define the **Parts**.

## Learning Objectives
- **The Vocabulary**: What is a "Layer", "Module", "Parameter", and "Activation"?
- **The Intuition**: Layers as "Filters" or "Assembly Line Stations".
- **The Mechanism**: `nn.Module` and the `forward()` method.
- **The Deep Dive**: What are "Parameters" and where do they live?


In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

torch.manual_seed(42)

## Part 1: The Vocabulary (Definitions First)

A Neural Network is just a big function made of smaller functions. Here are the names of the parts:

### 1. Layer
- A reusable block of math that transforms data.
- Example: `nn.Linear`, `nn.Conv2d`.
- Analogy: A single machine in a factory.

### 2. Module (`nn.Module`)
- The base class for all neural network parts in PyTorch.
- Your entire model is a Module. A single layer is also a Module.
- Analogy: The "Blueprint" for the machine.

### 3. Parameter
- The internal numbers (Weights and Biases) that the model learns.
- These are the "knobs" the optimizer turns.
- Analogy: The settings on the machine.

### 4. Activation Function
- A non-linear function applied after a layer.
- Without this, a neural network is just one big linear regression.
- Example: ReLU, Sigmoid.
- Analogy: The "Spark" or "Decision" to fire.

## Part 2: The Intuition (The Assembly Line)

Imagine a car factory assembly line.

1. **Input**: Raw Steel (Data).
2. **Station 1 (Layer 1)**: Stamps steel into doors. (Transforms shape).
3. **Station 2 (Layer 2)**: Welds doors to frame. (Combines features).
4. **Station 3 (Layer 3)**: Paints the car. (Final polish).
5. **Output**: Finished Car (Prediction).

In PyTorch, this factory is a `nn.Module`. The stations are Layers. The conveyor belt is the `forward()` method.

In [ ]:
import numpy as np

# Create a simple visualization of network architecture
fig, ax = plt.subplots(figsize=(14, 7))

# Layer positions
layers = [
    {"name": "Input\nLayer", "neurons": 4, "x": 0.1, "color": "lightblue"},
    {"name": "Hidden\nLayer 1", "neurons": 6, "x": 0.35, "color": "lightgreen"},
    {"name": "Hidden\nLayer 2", "neurons": 4, "x": 0.6, "color": "lightyellow"},
    {"name": "Output\nLayer", "neurons": 2, "x": 0.85, "color": "lightcoral"}
]

# Draw neurons and connections
neuron_positions = {}

for layer_idx, layer in enumerate(layers):
    x_pos = layer["x"]
    n_neurons = layer["neurons"]
    y_positions = np.linspace(0.2, 0.8, n_neurons)
    
    neuron_positions[layer_idx] = []
    
    for neuron_idx, y_pos in enumerate(y_positions):
        # Draw neuron
        circle = plt.Circle((x_pos, y_pos), 0.03, color=layer["color"], 
                          ec='black', linewidth=2, zorder=3)
        ax.add_patch(circle)
        neuron_positions[layer_idx].append((x_pos, y_pos))
    
    # Add layer label
    ax.text(x_pos, 0.95, layer["name"], ha='center', fontsize=12, 
            fontweight='bold', bbox=dict(boxstyle='round', facecolor=layer["color"], alpha=0.8))
    
    # Draw connections to next layer
    if layer_idx < len(layers) - 1:
        for pos1 in neuron_positions[layer_idx]:
            for pos2 in neuron_positions[layer_idx + 1]:
                ax.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]], 
                       'gray', alpha=0.3, linewidth=0.5, zorder=1)

# Add arrows between layers
for i in range(len(layers) - 1):
    mid_x = (layers[i]["x"] + layers[i+1]["x"]) / 2
    ax.annotate('', xy=(layers[i+1]["x"] - 0.05, 0.5), 
               xytext=(layers[i]["x"] + 0.05, 0.5),
               arrowprops=dict(arrowstyle='->', lw=3, color='blue'))
    
    # Add operation label
    if i == 0:
        operation = "Linear + ReLU"
    elif i == 1:
        operation = "Linear + ReLU"
    else:
        operation = "Linear"
    
    ax.text(mid_x, 0.05, operation, ha='center', fontsize=10,
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('Neural Network Architecture: Data Flow', fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("Data Flow:")
print("1. Input (4 features) → Linear transformation → 6 hidden neurons → ReLU activation")
print("2. Hidden (6) → Linear transformation → 4 hidden neurons → ReLU activation")
print("3. Hidden (4) → Linear transformation → 2 output predictions")
print("\nEach connection represents a learnable weight parameter!")

### Visualization: Neural Network as Assembly Line

Let's visualize how data flows through layers in a neural network.

## Part 3: The Linear Layer (The Workhorse)

The most basic layer is `nn.Linear`. It performs the equation of a line (in N dimensions):

$$ y = xA^T + b $$

Where:
- $x$: Input features.
- $A$: Weights (The "Slope").
- $b$: Bias (The "Intercept").

It simply maps input points to output points via rotation and stretching.

In [ ]:
# Visualize linear transformation in 2D
torch.manual_seed(42)

# Create sample 2D points (spiral pattern)
theta = torch.linspace(0, 4*np.pi, 100)
r = torch.linspace(0.5, 2, 100)
x_input = torch.stack([r * torch.cos(theta), r * torch.sin(theta)], dim=1)

# Create a linear layer (2D -> 2D transformation)
linear_layer = nn.Linear(2, 2)
with torch.no_grad():
    # Set specific weights for a clear transformation
    linear_layer.weight = nn.Parameter(torch.tensor([[1.5, 0.5], [-0.3, 1.2]]))
    linear_layer.bias = nn.Parameter(torch.tensor([0.5, -0.3]))

# Transform the points
x_output = linear_layer(x_input)

# Visualize
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# Original points
ax1.scatter(x_input[:, 0].numpy(), x_input[:, 1].numpy(), 
           c=range(len(x_input)), cmap='viridis', s=30, alpha=0.7)
ax1.set_xlabel('Feature 1', fontsize=12)
ax1.set_ylabel('Feature 2', fontsize=12)
ax1.set_title('Input Data (Original)', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axis('equal')
ax1.set_xlim(-3, 3)
ax1.set_ylim(-3, 3)

# Show the transformation
ax2.scatter(x_input[:, 0].numpy(), x_input[:, 1].numpy(), 
           c=range(len(x_input)), cmap='viridis', s=30, alpha=0.3, label='Original')
ax2.scatter(x_output[:, 0].detach().numpy(), x_output[:, 1].detach().numpy(), 
           c=range(len(x_output)), cmap='plasma', s=30, alpha=0.7, label='Transformed')
ax2.set_xlabel('Feature 1', fontsize=12)
ax2.set_ylabel('Feature 2', fontsize=12)
ax2.set_title('Linear Transformation', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.axis('equal')
ax2.set_xlim(-3, 3)
ax2.set_ylim(-3, 3)

# Transformed points
ax3.scatter(x_output[:, 0].detach().numpy(), x_output[:, 1].detach().numpy(), 
           c=range(len(x_output)), cmap='plasma', s=30, alpha=0.7)
ax3.set_xlabel('Feature 1', fontsize=12)
ax3.set_ylabel('Feature 2', fontsize=12)
ax3.set_title('Output Data (Transformed)', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.axis('equal')
ax3.set_xlim(-3, 3)
ax3.set_ylim(-3, 3)

plt.tight_layout()
plt.show()

print("What Linear Layers Do:")
print("• Rotation: Change the orientation of data")
print("• Scaling: Stretch or compress along axes")
print("• Translation: Shift the data (via bias)")
print(f"\nWeight matrix:\n{linear_layer.weight.data}")
print(f"Bias vector: {linear_layer.bias.data}")

### Visualization: How Linear Layers Transform Data

Let's visualize what a linear layer does geometrically - it rotates, scales, and shifts points in space.

In [ ]:
# Create a Linear Layer
# Input: 3 features (e.g., Age, Height, Weight)
# Output: 1 feature (e.g., Life Expectancy)
layer = nn.Linear(in_features=3, out_features=1)

print("Weights (A):", layer.weight)
print("Bias (b):", layer.bias)

# Pass data through it
input_data = torch.tensor([[1.0, 2.0, 3.0]]) # Batch of 1 sample
output = layer(input_data)

print("Output:", output)

## Part 4: Activation Functions (The Spark)

Linear layers can only learn straight lines. But the world is curved.

To learn curves, we need **Non-Linearity**. We call these "Activation Functions".

Think of a biological neuron. It gathers signals. If the signal is strong enough, it **FIRES** (Action Potential). If not, it stays silent.

- **ReLU (Rectified Linear Unit)**: The most common. If x > 0, return x. If x < 0, return 0. (Like a switch).
- **Sigmoid**: Squashes numbers between 0 and 1. (Like a probability).

In [ ]:
# Comprehensive visualization of activation functions
x = torch.linspace(-5, 5, 200)
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
tanh = nn.Tanh()
leaky_relu = nn.LeakyReLU(0.1)

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. ReLU
ax = axes[0, 0]
ax.plot(x, relu(x), linewidth=3, color='blue', label='ReLU')
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)
ax.fill_between(x.numpy(), 0, relu(x).numpy(), alpha=0.2, color='blue')
ax.set_xlabel('Input (x)', fontsize=11)
ax.set_ylabel('Output', fontsize=11)
ax.set_title('ReLU: max(0, x)\n"The Switch" - Dead below 0', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# 2. Sigmoid
ax = axes[0, 1]
ax.plot(x, sigmoid(x), linewidth=3, color='green', label='Sigmoid')
ax.axhline(y=0.5, color='k', linestyle='--', alpha=0.3, label='Midpoint')
ax.axhline(y=0, color='r', linestyle=':', alpha=0.3)
ax.axhline(y=1, color='r', linestyle=':', alpha=0.3)
ax.set_xlabel('Input (x)', fontsize=11)
ax.set_ylabel('Output', fontsize=11)
ax.set_title('Sigmoid: 1/(1+e⁻ˣ)\n"The Probability" - Output [0,1]', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# 3. Tanh
ax = axes[0, 2]
ax.plot(x, tanh(x), linewidth=3, color='orange', label='Tanh')
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax.axhline(y=-1, color='r', linestyle=':', alpha=0.3)
ax.axhline(y=1, color='r', linestyle=':', alpha=0.3)
ax.set_xlabel('Input (x)', fontsize=11)
ax.set_ylabel('Output', fontsize=11)
ax.set_title('Tanh: (eˣ-e⁻ˣ)/(eˣ+e⁻ˣ)\n"Centered" - Output [-1,1]', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# 4. Leaky ReLU
ax = axes[1, 0]
ax.plot(x, leaky_relu(x), linewidth=3, color='purple', label='Leaky ReLU')
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)
ax.fill_between(x.numpy(), 0, leaky_relu(x).numpy(), alpha=0.2, color='purple')
ax.set_xlabel('Input (x)', fontsize=11)
ax.set_ylabel('Output', fontsize=11)
ax.set_title('Leaky ReLU: max(0.1x, x)\n"Allows small negative values"', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# 5. Comparison of all
ax = axes[1, 1]
ax.plot(x, relu(x), linewidth=2, label='ReLU', alpha=0.8)
ax.plot(x, sigmoid(x), linewidth=2, label='Sigmoid', alpha=0.8)
ax.plot(x, tanh(x), linewidth=2, label='Tanh', alpha=0.8)
ax.plot(x, leaky_relu(x), linewidth=2, label='Leaky ReLU', alpha=0.8)
ax.set_xlabel('Input (x)', fontsize=11)
ax.set_ylabel('Output', fontsize=11)
ax.set_title('Comparison of All Activations', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# 6. Why non-linearity matters
ax = axes[1, 2]
x_demo = torch.linspace(-2, 2, 100)
# Linear only (no activation)
linear_output = 2 * x_demo + 1
# With ReLU activation
nonlinear_output = relu(2 * x_demo + 1)

ax.plot(x_demo, linear_output, linewidth=2.5, label='Linear only (boring!)', color='gray', linestyle='--')
ax.plot(x_demo, nonlinear_output, linewidth=2.5, label='Linear + ReLU (learns curves!)', color='red')
ax.set_xlabel('Input (x)', fontsize=11)
ax.set_ylabel('Output', fontsize=11)
ax.set_title('Why We Need Activation Functions', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.text(0, -1.5, 'Without activation: just a straight line!\nWith activation: can learn complex patterns', 
        ha='center', fontsize=9, bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.tight_layout()
plt.show()

print("Activation Function Properties:")
print("• ReLU: Fast, but can 'die' (always output 0)")
print("• Sigmoid: Smooth, but gradients vanish for large |x|")
print("• Tanh: Zero-centered, but also suffers from vanishing gradients")
print("• Leaky ReLU: Fixes dying ReLU problem with small negative slope")

In [ ]:
# Visualize the parameters of our network
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Get parameters from the model
params_list = list(model.named_parameters())

# Plot Layer 1 weights
ax = axes[0, 0]
weights_layer1 = params_list[0][1].detach().numpy()
im1 = ax.imshow(weights_layer1, cmap='RdBu', aspect='auto', vmin=-1, vmax=1)
ax.set_xlabel('Input Features (10)', fontsize=11)
ax.set_ylabel('Hidden Neurons (5)', fontsize=11)
ax.set_title('Layer 1 Weights (5×10)\nEach row = one neuron\'s weights', fontsize=12, fontweight='bold')
plt.colorbar(im1, ax=ax, label='Weight Value')
for i in range(weights_layer1.shape[0]):
    for j in range(weights_layer1.shape[1]):
        text = ax.text(j, i, f'{weights_layer1[i, j]:.2f}',
                      ha="center", va="center", color="black", fontsize=7)

# Plot Layer 1 bias
ax = axes[0, 1]
bias_layer1 = params_list[1][1].detach().numpy()
ax.barh(range(len(bias_layer1)), bias_layer1, color='steelblue', edgecolor='black')
ax.set_ylabel('Hidden Neurons (5)', fontsize=11)
ax.set_xlabel('Bias Value', fontsize=11)
ax.set_title('Layer 1 Biases (5)\nShift each neuron\'s activation', fontsize=12, fontweight='bold')
ax.axvline(x=0, color='red', linestyle='--', linewidth=1)
ax.grid(True, alpha=0.3, axis='x')
for i, v in enumerate(bias_layer1):
    ax.text(v + 0.02, i, f'{v:.3f}', va='center', fontsize=9)

# Plot Layer 2 weights
ax = axes[1, 0]
weights_layer2 = params_list[2][1].detach().numpy()
im2 = ax.imshow(weights_layer2, cmap='RdBu', aspect='auto', vmin=-1, vmax=1)
ax.set_xlabel('Hidden Features (5)', fontsize=11)
ax.set_ylabel('Output Neurons (1)', fontsize=11)
ax.set_title('Layer 2 Weights (1×5)\nCombines hidden features', fontsize=12, fontweight='bold')
plt.colorbar(im2, ax=ax, label='Weight Value')
for i in range(weights_layer2.shape[0]):
    for j in range(weights_layer2.shape[1]):
        text = ax.text(j, i, f'{weights_layer2[i, j]:.2f}',
                      ha="center", va="center", color="black", fontsize=9)

# Plot parameter histogram
ax = axes[1, 1]
all_weights = torch.cat([p.flatten() for name, p in model.named_parameters() if 'weight' in name])
all_biases = torch.cat([p.flatten() for name, p in model.named_parameters() if 'bias' in name])

ax.hist(all_weights.detach().numpy(), bins=30, alpha=0.6, label='Weights', color='blue', edgecolor='black')
ax.hist(all_biases.detach().numpy(), bins=15, alpha=0.6, label='Biases', color='orange', edgecolor='black')
ax.set_xlabel('Parameter Value', fontsize=11)
ax.set_ylabel('Count', fontsize=11)
ax.set_title('Distribution of All Parameters', fontsize=12, fontweight='bold')
ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(f"• Layer 1: {weights_layer1.size} weights + {bias_layer1.size} biases = {weights_layer1.size + bias_layer1.size}")
print(f"• Layer 2: {weights_layer2.size} weights + {params_list[3][1].numel()} biases = {weights_layer2.size + params_list[3][1].numel()}")
print("\nThese are the 'knobs' gradient descent will tune during training!")

### Visualization: Network Parameters

Let's visualize the actual parameters (weights and biases) that the network learns.

## Part 5: Building the Factory (nn.Module)

To build a full network, we subclass `nn.Module`. We must define two things:

1. `__init__`: **Define the stations**. (Create the layers).
2. `forward`: **Define the conveyor belt**. (Connect the layers).

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Station 1: 10 inputs -> 5 hidden features
        self.layer1 = nn.Linear(10, 5)
        # Station 2: 5 hidden -> 1 output
        self.layer2 = nn.Linear(5, 1)
        # The Spark
        self.activation = nn.ReLU()

    def forward(self, x):
        # The Conveyor Belt
        x = self.layer1(x)      # Step 1
        x = self.activation(x)  # Step 2 (Non-linearity)
        x = self.layer2(x)      # Step 3
        return x

model = SimpleNet()
print(model)

## Part 6: The Deep Dive (Parameters)

Where does the "Knowledge" live?

It lives in the **Parameters** (Weights and Biases). PyTorch automatically tracks these for you because you used `nn.Linear`.

Let's inspect them.

In [ ]:
print("Model Parameters:")
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")

# Total parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal Learnable Parameters: {total_params}")

## Part 7: Advanced nn.Module Patterns (FAANG Interview Essentials)

### nn.Sequential, nn.ModuleList, nn.ModuleDict

PyTorch provides containers for organizing layers. Knowing when to use each is critical.

In [ ]:
# nn.Sequential - Simple Linear Stacking
sequential_model = nn.Sequential(
    nn.Linear(10, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1)
)
print("nn.Sequential - for simple layer stacking:")
print(sequential_model)

# nn.ModuleList - When you need to iterate/index layers dynamically
class DynamicNet(nn.Module):
    def __init__(self, num_layers=3):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Linear(10, 10) for _ in range(num_layers)
        ])
        self.activation = nn.ReLU()
    
    def forward(self, x):
        for layer in self.layers:
            x = self.activation(layer(x))
        return x

dynamic_model = DynamicNet(num_layers=4)
print("\nnn.ModuleList - for dynamic/looped architectures:")
print(f"Number of layers: {len(dynamic_model.layers)}")

# nn.ModuleDict - Named layer access (great for multi-task learning)
class MultiTaskNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared = nn.Linear(10, 32)
        self.heads = nn.ModuleDict({
            'classification': nn.Linear(32, 5),
            'regression': nn.Linear(32, 1),
            'ranking': nn.Linear(32, 3)
        })
    
    def forward(self, x, task='classification'):
        x = torch.relu(self.shared(x))
        return self.heads[task](x)

multi_task_model = MultiTaskNet()
print("\nnn.ModuleDict - for named access (multi-task, multi-head):")
for name, head in multi_task_model.heads.items():
    print(f"  Task '{name}': output size {head.out_features}")

# CRITICAL: Why not use regular Python list/dict?
print("\n⚠️ CRITICAL: Always use nn.ModuleList/Dict, NOT regular Python list/dict!")
print("  - nn.ModuleList: Parameters are registered and tracked")
print("  - Python list: Parameters are INVISIBLE to optimizer!")

# Demo the difference
class BrokenNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = [nn.Linear(10, 10) for _ in range(3)]  # WRONG!
        
broken = BrokenNet()
print(f"\nBroken (Python list): {sum(p.numel() for p in broken.parameters())} parameters tracked")
print(f"Correct (ModuleList): {sum(p.numel() for p in dynamic_model.parameters())} parameters tracked")

### Weight Initialization - Critical for Training Stability

Poor initialization = slow training or no convergence. Know these methods:

In [ ]:
# Weight Initialization Methods

# 1. Xavier/Glorot - For sigmoid/tanh activations
def xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# 2. He/Kaiming - For ReLU activations (MOST COMMON)
def he_init(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# 3. Normal/Uniform with specific values
def custom_init(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0, std=0.01)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

# Apply initialization to a model
model = nn.Sequential(
    nn.Linear(10, 50),
    nn.ReLU(),
    nn.Linear(50, 1)
)

print("Before initialization:")
print(f"  Layer 1 weight std: {model[0].weight.std():.4f}")

model.apply(he_init)
print("\nAfter He initialization (for ReLU):")
print(f"  Layer 1 weight std: {model[0].weight.std():.4f}")

# Visualize initialization comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

init_methods = [
    ('Xavier (Uniform)', lambda m: nn.init.xavier_uniform_(m.weight) if isinstance(m, nn.Linear) else None),
    ('He/Kaiming (Normal)', lambda m: nn.init.kaiming_normal_(m.weight) if isinstance(m, nn.Linear) else None),
    ('Normal(0, 1)', lambda m: nn.init.normal_(m.weight, 0, 1) if isinstance(m, nn.Linear) else None)
]

for ax, (name, init_fn) in zip(axes, init_methods):
    layer = nn.Linear(100, 100)
    init_fn(layer)
    weights = layer.weight.detach().flatten().numpy()
    ax.hist(weights, bins=50, density=True, alpha=0.7, edgecolor='black')
    ax.set_title(f'{name}\nstd={weights.std():.3f}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Weight Value')
    ax.set_ylabel('Density')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInitialization Guidelines:")
print("• Xavier: Keeps variance stable through sigmoid/tanh networks")
print("• He/Kaiming: Accounts for ReLU killing half the activations")
print("• Normal(0, 1): Often TOO large, causes saturation/explosion")
print("\n🔑 FAANG Tip: Use He init for ReLU networks, Xavier for sigmoid/tanh")

### Model Analysis - Parameter Counting and Layer Freezing

Essential for transfer learning, debugging, and understanding model complexity.

In [ ]:
# Model Analysis Utilities

def count_parameters(model, trainable_only=True):
    """Count parameters in a model."""
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    return sum(p.numel() for p in model.parameters())

def model_summary(model, input_size=None):
    """Print a summary of model layers and parameters."""
    print(f"{'Layer Name':<30} {'Output Shape':<20} {'Param #':<15}")
    print("-" * 65)
    
    total_params = 0
    trainable_params = 0
    
    for name, module in model.named_modules():
        if len(list(module.children())) == 0:  # Leaf module
            params = sum(p.numel() for p in module.parameters())
            trainable = sum(p.numel() for p in module.parameters() if p.requires_grad)
            total_params += params
            trainable_params += trainable
            
            if params > 0:
                print(f"{name:<30} {str(list(module.parameters())[0].shape):<20} {params:<15,}")
    
    print("-" * 65)
    print(f"Total params: {total_params:,}")
    print(f"Trainable params: {trainable_params:,}")
    print(f"Non-trainable params: {total_params - trainable_params:,}")

# Demo model
demo_model = nn.Sequential(
    nn.Linear(784, 256),    # 784*256 + 256 = 200,960
    nn.ReLU(),
    nn.Linear(256, 128),    # 256*128 + 128 = 32,896
    nn.ReLU(),
    nn.Linear(128, 10)      # 128*10 + 10 = 1,290
)

print("Model Summary:")
model_summary(demo_model)

# Layer Freezing for Transfer Learning
print("\n--- Layer Freezing for Transfer Learning ---")

# Freeze all layers
for param in demo_model.parameters():
    param.requires_grad = False

print(f"After freezing all: {count_parameters(demo_model)} trainable params")

# Unfreeze only the last layer (fine-tuning)
for param in demo_model[4].parameters():  # Last Linear layer
    param.requires_grad = True

print(f"After unfreezing last layer: {count_parameters(demo_model)} trainable params")

# Common pattern: Freeze backbone, train head
class TransferLearningModel(nn.Module):
    def __init__(self, pretrained_backbone, num_classes):
        super().__init__()
        self.backbone = pretrained_backbone
        
        # Freeze backbone
        for param in self.backbone.parameters():
            param.requires_grad = False
        
        # New trainable head
        self.head = nn.Linear(128, num_classes)
    
    def forward(self, x):
        # Features from frozen backbone
        with torch.no_grad():  # Extra safety, no gradient storage
            features = self.backbone(x)
        return self.head(features)

print("\n🔑 FAANG Tip: Freeze early layers (generic features), fine-tune later layers (task-specific)")

## Part 8: FAANG Interview Questions - Neural Network Architecture

### Question 1: "Explain the difference between nn.Module's __init__ and forward methods"

**Answer**:
- `__init__`: Called once at instantiation. Define layers, buffers, and submodules.
- `forward`: Called on every input. Defines the computation graph dynamically.

```python
class MyModel(nn.Module):
    def __init__(self):  # Define WHAT layers exist
        super().__init__()
        self.layer1 = nn.Linear(10, 5)
        
    def forward(self, x):  # Define HOW data flows
        return self.layer1(x)
```

Key insight: PyTorch builds computation graph during `forward()`, enabling dynamic architectures.

### Question 2: "Why use ReLU instead of Sigmoid for hidden layers?"

**Answer**:
| Property | ReLU | Sigmoid |
|----------|------|---------|
| Range | [0, ∞) | (0, 1) |
| Gradient | 0 or 1 | 0 to 0.25 |
| Computation | Fast (max) | Slow (exp) |
| Vanishing Gradient | Only for x<0 | For |x|>2 |

ReLU advantages:
1. **Sparse activation**: Only ~50% of neurons fire
2. **No vanishing gradient** for positive values
3. **Faster computation**: Just a max operation
4. **Faster convergence**: ~6x faster than sigmoid

### Question 3: "What's the purpose of Batch Normalization?"

**Answer**:
Batch Norm normalizes layer inputs to have mean=0, std=1, then applies learnable scale/shift.

```python
# BatchNorm does:
# 1. Compute batch mean and variance
# 2. Normalize: (x - mean) / sqrt(var + eps)
# 3. Scale and shift: gamma * normalized + beta
```

Benefits:
1. **Faster training**: Allows higher learning rates
2. **Regularization**: Acts as noise (like dropout)
3. **Reduces internal covariate shift**: Stabilizes layer inputs
4. **Less sensitive to initialization**: More forgiving

### Question 4: "Explain the difference between model.train() and model.eval()"

**Answer**:
```python
model.train()  # Training mode
model.eval()   # Evaluation mode
```

These modes affect:
1. **Dropout**: Active in train, disabled in eval
2. **BatchNorm**: Uses batch stats in train, running stats in eval
3. **Gradient behavior**: Unchanged (use `torch.no_grad()` separately)

Common mistake: Forgetting to call `model.eval()` before inference!

### Question 5: "How would you implement skip connections (residual connections)?"

**Answer**:
```python
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.bn2 = nn.BatchNorm2d(channels)
        
    def forward(self, x):
        residual = x  # Save input
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual  # Add skip connection!
        return F.relu(out)
```

Why skip connections work:
1. **Gradient highway**: Gradients flow directly to early layers
2. **Identity mapping**: Easy to learn "do nothing"
3. **Enables very deep networks**: 100+ layers possible

## Summary: Neural Network Building Checklist

### Foundation (Know These Cold)
- [ ] `nn.Module` is the base class for all PyTorch models
- [ ] `__init__` defines layers, `forward` defines data flow
- [ ] Parameters are automatically tracked when using nn layers
- [ ] Call `super().__init__()` in your module's `__init__`

### Layers (Daily Usage)
- [ ] `nn.Linear(in, out)` - Fully connected layer
- [ ] `nn.Conv2d(in_ch, out_ch, kernel)` - Convolutional layer
- [ ] `nn.BatchNorm2d(channels)` - Batch normalization
- [ ] `nn.Dropout(p)` - Regularization via random zeroing

### Activations (Interview Favorites)
- [ ] ReLU - Default choice for hidden layers (fast, no vanishing gradient)
- [ ] Sigmoid - Output for binary classification
- [ ] Softmax - Output for multi-class classification
- [ ] GELU/SiLU - Modern transformers

### Containers (Architecture Patterns)
- [ ] `nn.Sequential` - Simple layer stacking
- [ ] `nn.ModuleList` - Dynamic/looped architectures
- [ ] `nn.ModuleDict` - Named access (multi-task)

### Advanced Topics (FAANG Level)
- [ ] Weight initialization (Xavier for tanh, He for ReLU)
- [ ] Layer freezing for transfer learning
- [ ] Parameter counting and model analysis
- [ ] Skip connections for deep networks

---
**Next**: Notebook 03 - Training Your First Model (The training loop)